# Getting Started with OKX Client Gateway

This notebook demonstrates how to use the `okx-client-gw-py` library to interact with the OKX exchange API.

## Prerequisites

1. Install the package: `pip install okx-client-gw`
2. Sign up for an OKX account: [Create account](https://www.okx.com/account/register)
3. Create API keys: [Create API Key](https://www.okx.com/account/my-api)

**Note**: For demo trading, use the OKX demo trading environment. Set your credentials in environment variables:
- `OKX_API_KEY`
- `OKX_SECRET_KEY`
- `OKX_PASSPHRASE`

## Setup and Imports

The library follows clean architecture principles with services for different API domains.

In [ ]:
from decimal import Decimal

# Core imports
from okx_client_gw import OkxHttpClient, OkxConfig
from okx_client_gw.core.auth import OkxCredentials

# Services
from okx_client_gw.application.services import (
    MarketDataService,
    InstrumentService,
    AccountService,
    TradeService,
    PublicDataService,
)

# Domain models and enums
from okx_client_gw.domain.enums import InstType, TradeSide, TradeMode, OrderType

## Configuration

Load credentials from environment variables. For demo trading, use `demo=True`.

In [ ]:
# Load credentials from environment variables
# Set OKX_API_KEY, OKX_SECRET_KEY, OKX_PASSPHRASE before running
try:
    credentials = OkxCredentials.from_env()
    print("✓ Credentials loaded from environment")
except ValueError as e:
    print(f"⚠ No credentials found: {e}")
    print("  Some examples require authentication. Set environment variables:")
    print("  - OKX_API_KEY")
    print("  - OKX_SECRET_KEY")
    print("  - OKX_PASSPHRASE")
    credentials = None

# Configure for demo trading
config = OkxConfig(use_demo=True)  # Set demo=False for live trading

## Part 1: Public Data (No Authentication Required)

These operations don't require API credentials.

### Get Market Tickers

Fetch current market prices for all spot trading pairs.

In [ ]:
async def get_spot_tickers():
    """Get all spot market tickers."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        tickers = await service.get_tickers(InstType.SPOT)
        
        print(f"Found {len(tickers)} spot trading pairs\n")
        
        # Show top 5 by volume
        sorted_tickers = sorted(tickers, key=lambda t: float(t.vol_ccy_24h or 0), reverse=True)
        print("Top 5 by 24h volume:")
        for ticker in sorted_tickers[:5]:
            print(f"  {ticker.inst_id}: ${float(ticker.last):,.2f} (vol: ${float(ticker.vol_ccy_24h):,.0f})")
        
        return tickers

tickers = await get_spot_tickers()

### Get Available Instruments

Fetch information about available trading instruments.

In [ ]:
async def get_spot_instruments():
    """Get all spot instruments."""
    async with OkxHttpClient(config=config) as client:
        service = InstrumentService(client)
        instruments = await service.get_instruments(InstType.SPOT)
        
        print(f"Found {len(instruments)} spot instruments\n")
        
        # Show BTC pairs
        btc_instruments = [i for i in instruments if i.base_ccy == "BTC"]
        print(f"BTC trading pairs ({len(btc_instruments)}):")
        for inst in btc_instruments[:5]:
            print(f"  {inst.inst_id}: min size={inst.min_sz}, tick size={inst.tick_sz}")
        
        return instruments

instruments = await get_spot_instruments()

### Get Single Ticker

Fetch ticker for a specific instrument.

In [ ]:
async def get_btc_ticker():
    """Get BTC-USDT ticker details."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        ticker = await service.get_ticker("BTC-USDT")
        
        print("BTC-USDT Ticker:")
        print(f"  Last Price: ${float(ticker.last):,.2f}")
        print(f"  24h High:   ${float(ticker.high_24h):,.2f}")
        print(f"  24h Low:    ${float(ticker.low_24h):,.2f}")
        print(f"  24h Volume: {float(ticker.vol_24h):,.2f} BTC")
        print(f"  24h Change: {float(ticker.change_24h) * 100:.2f}%")
        print(f"  Bid:        ${float(ticker.bid_px):,.2f}")
        print(f"  Ask:        ${float(ticker.ask_px):,.2f}")
        print(f"  Spread:     ${float(ticker.ask_px) - float(ticker.bid_px):.2f}")
        
        return ticker

btc_ticker = await get_btc_ticker()

### Get Order Book

Fetch the order book for a trading pair.

In [ ]:
async def get_order_book():
    """Get BTC-USDT order book."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        orderbook = await service.get_orderbook("BTC-USDT", depth=10)
        
        print("BTC-USDT Order Book:")
        print(f"  Timestamp: {orderbook.ts}")
        print(f"  Spread: ${orderbook.spread:.2f}")
        print(f"  Mid Price: ${orderbook.mid_price:.2f}")
        print()
        print("  Top 5 Asks (sells):")
        for level in orderbook.asks[:5]:
            print(f"    ${float(level.price):,.2f} x {float(level.size):.4f}")
        print()
        print("  Top 5 Bids (buys):")
        for level in orderbook.bids[:5]:
            print(f"    ${float(level.price):,.2f} x {float(level.size):.4f}")
        
        return orderbook

orderbook = await get_order_book()

### Get Recent Trades

Fetch recent trade history for a trading pair.

In [ ]:
async def get_recent_trades():
    """Get recent BTC-USDT trades."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        trades = await service.get_trades("BTC-USDT", limit=10)
        
        print(f"Recent BTC-USDT Trades ({len(trades)}):")
        for trade in trades[:5]:
            side = "BUY " if trade.side == "buy" else "SELL"
            print(f"  {trade.ts.strftime('%H:%M:%S')} {side} {float(trade.sz):.6f} @ ${float(trade.px):,.2f}")
        
        return trades

trades = await get_recent_trades()

### Get Candlestick Data

Fetch OHLCV candlestick data for charting.

In [ ]:
async def get_candles():
    """Get BTC-USDT 1-hour candles."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        candles = await service.get_candles("BTC-USDT", bar="1H", limit=24)
        
        print(f"Last 24 hourly candles for BTC-USDT:")
        print()
        for candle in candles[:5]:
            change = ((candle.close - candle.open) / candle.open) * 100
            direction = "↑" if change >= 0 else "↓"
            print(f"  {candle.timestamp.strftime('%Y-%m-%d %H:%M')} O:{float(candle.open):,.0f} H:{float(candle.high):,.0f} L:{float(candle.low):,.0f} C:{float(candle.close):,.0f} {direction}{abs(change):.2f}%")
        print("  ...")
        
        return candles

candles = await get_candles()

## Part 2: Extended Public Data

Additional public data endpoints for currencies, discount rates, and funding rates.

### Get Available Currencies

In [ ]:
async def get_currencies():
    """Get available currencies."""
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        service = PublicDataService(client)
        currencies = await service.get_currencies()
        
        print(f"Available currencies: {len(currencies)}")
        print()
        # Show first 10
        for ccy in currencies[:10]:
            print(f"  {ccy.ccy}: {ccy.name} (chain: {ccy.chain or 'N/A'})")
        print("  ...")
        
        return currencies

currencies = await get_currencies()

### Get Funding Rates (Perpetual Swaps)

In [ ]:
async def get_funding_rates():
    """Get funding rates for perpetual swaps."""
    async with OkxHttpClient(config=config) as client:
        service = PublicDataService(client)
        
        # Get current funding rate for BTC-USDT-SWAP
        rate = await service.get_funding_rate("BTC-USDT-SWAP")
        annualized = await service.get_funding_rate_annualized("BTC-USDT-SWAP")
        
        print("BTC-USDT-SWAP Funding Rate:")
        print(f"  Current Rate:    {float(rate.funding_rate) * 100:.4f}%")
        print(f"  Annualized:      {annualized * 100:.2f}%")
        print(f"  Funding Time:    {rate.funding_time}")
        if rate.next_funding_rate:
            print(f"  Next Rate:       {float(rate.next_funding_rate) * 100:.4f}%")
        if rate.next_funding_time:
            print(f"  Next Time:       {rate.next_funding_time}")
        
        return rate

funding_rate = await get_funding_rates()

## Part 3: Authenticated Operations (Requires API Key)

These operations require valid API credentials. Make sure you've set the environment variables.

### Check Account Balance

In [ ]:
async def get_account_balance():
    """Get account balance (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        service = AccountService(client)
        balance = await service.get_balance()
        
        print("Account Balance:")
        print(f"  Total Equity:   ${float(balance.total_eq):,.2f}")
        print(f"  Available:      ${float(balance.adj_eq):,.2f}")
        if balance.mgn_ratio:
            print(f"  Margin Ratio:   {float(balance.mgn_ratio) * 100:.1f}%")
        print(f"  Account Health: {'✓ Healthy' if balance.is_healthy else '⚠ Low margin'}")
        print()
        
        if balance.details:
            print("  Currency Breakdown:")
            for detail in balance.details[:5]:
                if float(detail.eq) > 0:
                    print(f"    {detail.ccy}: {float(detail.eq):.8f} (${float(detail.eq_usd):,.2f})")
        
        return balance

balance = await get_account_balance()

### Get Account Configuration

In [ ]:
async def get_account_config():
    """Get account configuration (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        service = AccountService(client)
        config_data = await service.get_config()
        
        print("Account Configuration:")
        print(f"  Account Mode:   {config_data.account_mode_name}")
        print(f"  Position Mode:  {config_data.pos_mode}")
        print(f"  Auto Loan:      {'Yes' if config_data.auto_loan else 'No'}")
        print(f"  Greeks Type:    {config_data.greeks_type}")
        print(f"  UID:            {config_data.uid}")
        print(f"  Main UID:       {config_data.main_uid}")
        
        # Explain account mode
        print()
        if config_data.acct_lv == "1":
            print("  → Simple mode: Basic spot trading only")
        elif config_data.acct_lv == "2":
            print("  → Single-currency margin: Trade with leverage using one currency")
        elif config_data.acct_lv == "3":
            print("  → Multi-currency margin: Use multiple currencies as collateral")
        elif config_data.acct_lv == "4":
            print("  → Portfolio margin: Advanced cross-margining across positions")
        
        return config_data

account_config = await get_account_config()

### Get Maximum Available Trading Size

In [ ]:
async def get_max_size():
    """Get maximum available trading size (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        service = AccountService(client)
        result = await service.get_max_available_size(
            inst_id="BTC-USDT",
            td_mode="cash"  # or "cross", "isolated" for margin
        )
        
        print("Maximum Available Size for BTC-USDT (cash mode):")
        print(f"  Max Buy:  {result.get('maxBuy', 'N/A')} BTC")
        print(f"  Max Sell: {result.get('maxSell', 'N/A')} BTC")
        
        return result

max_size = await get_max_size()

## Part 4: Trading Operations (Requires API Key)

**⚠ WARNING**: These operations will place real orders (in demo mode). Review carefully before running!

### Place a Limit Order

Place a limit buy order for BTC-USDT at a price well below market.

In [ ]:
async def place_limit_order():
    """Place a limit order (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        # Get current price to set limit below market
        market_service = MarketDataService(client)
        ticker = await market_service.get_ticker("BTC-USDT")
        limit_price = Decimal(str(float(ticker.last) * 0.9))  # 10% below market
        
        trade_service = TradeService(client)
        result = await trade_service.place_limit_order(
            inst_id="BTC-USDT",
            side=TradeSide.BUY,
            sz=Decimal("0.001"),  # Small size for demo
            px=limit_price,
            td_mode=TradeMode.CASH,
        )
        
        if result.get("sCode") == "0":
            print("✓ Limit order placed successfully!")
            print(f"  Order ID:    {result.get('ordId')}")
            print(f"  Instrument:  BTC-USDT")
            print(f"  Side:        BUY")
            print(f"  Price:       ${float(limit_price):,.2f}")
            print(f"  Size:        0.001 BTC")
        else:
            print(f"✗ Order failed: {result.get('sMsg')}")
        
        return result

# Uncomment to execute:
# order_result = await place_limit_order()

### Place a Market Order

In [ ]:
async def place_market_order():
    """Place a market order (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        trade_service = TradeService(client)
        
        # Market buy with quote currency (USDT)
        result = await trade_service.place_market_order(
            inst_id="BTC-USDT",
            side=TradeSide.BUY,
            sz=Decimal("10"),  # 10 USDT worth
            td_mode=TradeMode.CASH,
            tgt_ccy="quote_ccy",  # Size in quote currency (USDT)
        )
        
        if result.get("sCode") == "0":
            print("✓ Market order placed successfully!")
            print(f"  Order ID: {result.get('ordId')}")
        else:
            print(f"✗ Order failed: {result.get('sMsg')}")
        
        return result

# Uncomment to execute:
# market_order = await place_market_order()

### Get Pending Orders

In [ ]:
async def get_pending_orders():
    """Get pending orders (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        trade_service = TradeService(client)
        orders = await trade_service.get_pending_orders()
        
        if orders:
            print(f"Pending Orders ({len(orders)}):")
            for order in orders:
                print(f"  {order.ord_id}: {order.inst_id} {order.side.value} {order.sz} @ {order.px}")
        else:
            print("No pending orders")
        
        return orders

pending = await get_pending_orders()

### Get Order Details

In [ ]:
async def get_order_details(ord_id: str):
    """Get order details (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        trade_service = TradeService(client)
        order = await trade_service.get_order("BTC-USDT", ord_id=ord_id)
        
        print(f"Order Details:")
        print(f"  Order ID:    {order.ord_id}")
        print(f"  Instrument:  {order.inst_id}")
        print(f"  Side:        {order.side.value}")
        print(f"  Type:        {order.ord_type.value}")
        print(f"  Price:       {order.px}")
        print(f"  Size:        {order.sz}")
        print(f"  Filled:      {order.fill_sz}")
        print(f"  State:       {order.state.value}")
        print(f"  Created:     {order.c_time}")
        
        return order

# Replace with actual order ID:
# order_details = await get_order_details("your-order-id-here")

### Cancel an Order

In [ ]:
async def cancel_order(ord_id: str):
    """Cancel an order (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        trade_service = TradeService(client)
        result = await trade_service.cancel_order("BTC-USDT", ord_id=ord_id)
        
        if result.get("sCode") == "0":
            print(f"✓ Order {ord_id} cancelled successfully")
        else:
            print(f"✗ Cancel failed: {result.get('sMsg')}")
        
        return result

# Replace with actual order ID:
# cancel_result = await cancel_order("your-order-id-here")

### Amend an Order

In [ ]:
async def amend_order(ord_id: str, new_price: str = None, new_size: str = None):
    """Amend an existing order (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        trade_service = TradeService(client)
        result = await trade_service.amend_order(
            inst_id="BTC-USDT",
            ord_id=ord_id,
            new_px=new_price,
            new_sz=new_size,
        )
        
        if result.get("sCode") == "0":
            print(f"✓ Order {ord_id} amended successfully")
        else:
            print(f"✗ Amend failed: {result.get('sMsg')}")
        
        return result

# Replace with actual order ID:
# amend_result = await amend_order("your-order-id-here", new_price="19500")

### Get Order History

In [ ]:
async def get_order_history():
    """Get order history (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        trade_service = TradeService(client)
        orders = await trade_service.get_order_history(
            inst_type=InstType.SPOT,
            limit=10
        )
        
        if orders:
            print(f"Order History (last {len(orders)} orders):")
            for order in orders:
                status = "✓" if order.state.value == "filled" else "✗" if order.state.value == "canceled" else "○"
                print(f"  {status} {order.c_time.strftime('%Y-%m-%d %H:%M')} {order.inst_id} {order.side.value} {order.sz} @ {order.px or 'market'} [{order.state.value}]")
        else:
            print("No order history")
        
        return orders

history = await get_order_history()

### Cancel All Pending Orders

In [ ]:
async def cancel_all_orders():
    """Cancel all pending orders for BTC-USDT (requires authentication)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        trade_service = TradeService(client)
        results = await trade_service.cancel_all_orders("BTC-USDT")
        
        if results:
            print(f"Cancelled {len(results)} orders")
            for result in results:
                print(f"  Order {result.get('ordId')}: {result.get('sMsg', 'OK')}")
        else:
            print("No orders to cancel")
        
        return results

# Uncomment to execute:
# cancel_all = await cancel_all_orders()

## Summary

This notebook demonstrated the core functionality of `okx-client-gw-py`:

**Public Data (No Auth Required)**:
- `MarketDataService`: Tickers, order books, trades, candles
- `InstrumentService`: Available trading instruments
- `PublicDataService`: Currencies, funding rates

**Private Data (Auth Required)**:
- `AccountService`: Balance, positions, configuration
- `TradeService`: Order placement, cancellation, history

**Key Features**:
- Async/await pattern for efficient I/O
- Strong typing with Pydantic models
- Clean service-based architecture
- Environment-based credential management

For more advanced usage including derivatives trading, see `02_trading_derivatives.ipynb`.